In [47]:
import matplotlib.pyplot as plt
from math import ceil
from os import listdir
import shutil, os
from os.path import isfile, join
from datetime import datetime
import numpy as np
import pandas as pd
import time
import cv2
import ast
TrDict = {"mil": cv2.TrackerMIL_create,
        "medianflow": cv2.TrackerMedianFlow_create}

# SETTING VERSION OF TRACKER ALGORITHM

In [58]:
# TRACKING_ALGORITHM = 'mil'
TRACKING_ALGORITHM = 'medianflow'

# CREATE A COPY OF DETECTED VORTICES FOLDER FOR MODIFICATIONS

In [54]:
vortices_start = 140.0
vortices_end = 200.0

In [55]:
from_path = f'../CVDetection/BestDetectorResults-Method5/detected_vortices'
to_path = './detected_vortices_copy/'
os.mkdir(to_path)

for t in np.round(np.arange(vortices_start, vortices_end+0.1, 0.1),1):
    f = f'vortices_time={np.round(t, 1)}0.csv'
    shutil.copy(join(from_path, f), to_path)

# TRACKING PROCESS

In [56]:
time_range = np.round(np.arange(vortices_start, vortices_end-40+0.1, 0.1),1)
bbox_size = 24 # size of tracker box
no_of_vortices = 0
now = datetime.now()
program_starts = time.time()

for start in time_range:
    
    ''' PART I '''
    ''' Tracker frames creation for t = start '''
    cap = cv2.VideoCapture('Original_Movies/X_phase_velocity_evolution_3_cycles_F=0.06.mkv')
    start_frame_number = round((start-0.1)*10)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number) 
    t = 0.1
    t = t * (start_frame_number+1)
    trackers = cv2.MultiTracker_create()
    vortex_trajectory = {}
    previous_tracker_frame = {}

    while True:
        ret, frame = cap.read()
        roi = frame[149:1396, 104:1351]
        roi = cv2.resize(roi,(817,817))   
        
        ''' READING DATA '''
        vortices_coordinates = pd.read_csv(f'detected_vortices_copy/vortices_time={np.round(t, 1)}0.csv', delimiter='\t', index_col='Vortex')

        k = len(vortices_coordinates)

        ''' BORDER CONDITIONS '''
        for i in range(k):
            cx = vortices_coordinates.iloc[i]['x']
            if cx < 11:
                cx = 11
            elif cx > 805:
                cx = 805
            cy = vortices_coordinates.iloc[i]['y']
            if cy < 11:
                cy = 11
            elif cy > 805:
                cy = 805
            bbi = (cx-(bbox_size/2), cy-(bbox_size/2), bbox_size, bbox_size) # cv2.selectROI('Frame', roi)
            vortex_trajectory[i+no_of_vortices] = []
            tracker_i = TrDict[TRACKING_ALGORITHM]()
            trackers.add(tracker_i, roi, bbi)
        break

        t += 0.1
    cap.release()
    
    
    ''' PART II '''
    ''' Evolving of vortices '''
    cap = cv2.VideoCapture('Original_Movies/X_phase_velocity_evolution_3_cycles_F=0.06.mkv')
    start_frame_number = round((start-0.1)*10)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_number)
    t = 0.1
    t = t * (start_frame_number+1)
    inactive_trackers = []
    coordinates_to_remove = {}
    map_overlap = {}
    first_deactivation = False

    while True:
        ret, frame = cap.read()
        roi = frame[149:1396, 104:1351]
        roi = cv2.resize(roi,(817,817))    
  
        ''' Trackers update '''
        if not ret:
            break
        (success, boxes) = trackers.update(roi)


        ''' READING DATA '''
        current_vortices = []
        vortices_coordinates = pd.read_csv(f'detected_vortices_copy/vortices_time={np.round(t, 1)}0.csv', delimiter='\t', index_col='Vortex')
        k = len(vortices_coordinates)
        for i in range(k):
            cx = vortices_coordinates.iloc[i]['x']
            cy = vortices_coordinates.iloc[i]['y']
            current_vortices.append((cx, cy))
        
        
        ''' MORE VORTICES AT GIVEN MOMENT OF TIME - SMALLER DISTANCE BETWEEN TRACKERS '''
        if k > 30:
            distance = 10
        else:
            distance = 20

        ''' FOR CHECKING OVERLAPPING '''
        map_overlap = {}
        for (x,y) in current_vortices:
            map_overlap[(x,y)] = []    
        
            
        ''' Trackers analysis '''
        for (i, box) in enumerate(boxes):
            if i not in set(inactive_trackers):
                (x,y,w,h) = [int(a) for a in box]

                flag = False
                
                ''' MODIFICATION - CALCULATE ALL THE DISTANCES BETWEEN TRACKER FRAME AND VORTICES COORDINATES '''
                ''' FIND MIN DISTANCE AND CHECK IF IT IS BELOW THRESHOLD '''
                
                b = np.array([x+(int(w/2.)), y+(int(h/2.))])
                ''' calculate distances '''
                distances = {}
                for vortex in current_vortices:
                    a = np.array([vortex[0], vortex[1]])
                    distances[(vortex[0], vortex[1])] = np.linalg.norm(a - b)
                
                
                if len(distances) > 0:
                    ''' find minimum '''
                    min_distance_key = min(distances, key=distances.get)
                
                    ''' check if minimum is below distance threshold and add to trajectory'''
                    if distances[min_distance_key] < distance:
                        flag = True
                        vortex_trajectory[i+no_of_vortices].append([np.round(t, 1), min_distance_key[0], min_distance_key[1]])
                        ''' For overlapping check '''
                        map_overlap[(min_distance_key[0], min_distance_key[1])].append((i, distances[min_distance_key]))
                        
                        if np.round(t, 1) not in coordinates_to_remove.keys():
                            coordinates_to_remove[np.round(t, 1)] = []
                            coordinates_to_remove[np.round(t, 1)].append([min_distance_key[0], min_distance_key[1]])
                        else:
                            coordinates_to_remove[np.round(t, 1)].append([min_distance_key[0], min_distance_key[1]])
                            
                if not flag:
                    ''' Tracker deactivation '''
                    inactive_trackers.append(i)
                    first_deactivation = True
                    
            ''' overlap check '''        
            for k, v in map_overlap.items():
                if len(v) > 1:
                    min_tuple = min(v, key=lambda t: t[1])
                    for vrtx, dist in [x for x in v if x is not min_tuple]:
                        inactive_trackers.append(vrtx)
   
        
        ''' CONDITION - ALL THE VORTICES ARE DEAD - STOP TRACKING THE EVOLUTION '''
        if len(set(inactive_trackers)) == len(trackers.getObjects()):
            break

#             cv2.imshow('Frame', roi)  


        if t > vortices_end:
            break
        t += 0.1

        key = cv2.waitKey(30)
        if key == 27:
            break

    cap.release()    
    
    no_of_vortices += len(trackers.getObjects())
    
    ''' Part of code for vortices trajectories analysis '''

    for m in coordinates_to_remove.keys():
        df = pd.read_csv(f'detected_vortices_copy/vortices_time={m}0.csv', delimiter='\t', index_col='Vortex')
        df_coord = pd.DataFrame(data=coordinates_to_remove[m], columns=['x', 'y'])
        df_coord.index.name = 'Vortex'
        df = pd.concat([df,df_coord]).drop_duplicates(keep=False)
        df.to_csv(f'detected_vortices_copy/vortices_time={m}0.csv', sep='\t')
          
    
    ''' SAVING TO FILE TRAJECTORIES '''
    
    with open(f'Tracking_Results/trajectories_RawImage_{TRACKING_ALGORITHM}_{now.strftime("%Y-%m-%d_%H-%M-%S")}.csv', 'a') as f:
        for k,v in vortex_trajectory.items():
            f.write(f'{k}: {v},\n')
    
    if start % 5 == 0:
        print("Checkpoint:", start)
    
cap.release()
cv2.destroyAllWindows()

program_ends = time.time()
print("It has been {0} seconds since the program started".format(program_ends - program_starts))

Checkpoint: 140.0
Checkpoint: 145.0
Checkpoint: 150.0
Checkpoint: 155.0
Checkpoint: 160.0


# REMOVE COPY OF DETECTED VORTICES

In [57]:
shutil.rmtree(to_path)